In [1]:
from libs.connector.postgres_connector import PostgresToJsonConnector
from libs.handler.postgres_handler import PostgresHandler
from libs.handler.json_handler import JsonHandler
from libs.log_handler.log_handler import LogHandler

from libs.dedupe.algo.exact_name_url import deduplicate_exact_match_name_url
from libs.dataframes.to_types import to_list_of_dicts


import pandas as pd


log_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
)

logger = LogHandler(db_handler=log_handler)

input_qry = '''
    SELECT
        id,
        name,
        website
    FROM organization
    '''

input_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
    query = input_qry,
    schema = 'connect211_withinreach_hsds3',
    env_prefix = 'POSTGRES'
)

output_handler = JsonHandler()

print (input_handler)
print (output_handler)

Host: silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com
Port: 25060
Database: defaultdb
Username: doadmin
Schema: connect211_withinreach_hsds3
Connection Alive? Yes


In [2]:

conn = PostgresToJsonConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=logger,
)

In [3]:
# job_id = conn.mutate(deduplicate_exact_match_name_url)

In [4]:
# print (job_id)

In [5]:


DEDUPLICATION_STEPS = {
    'dedupe_io_names_addresses': 'libs.dedupe.ml.names_addresses.names_addresses',
    'deduplicate_exact_match_name_url': 'libs.dedupe.algo.exact_name_url',
}
steps = dict()
my_steps = ['deduplicate_exact_match_name_url']
for step in my_steps:
    if DEDUPLICATION_STEPS.get(step):
        steps[step] = DEDUPLICATION_STEPS[step]


job_id = conn.mutate(conn.parse_steps(steps))

In [6]:
print (job_id)

1bf8299d-ba62-42ed-a047-e8acda2a6d81
